## 15GB数据，运行时间15m

In [1]:
%load_ext cython

In [2]:
%%cython

cdef struct Rect:
    float lon_min
    float lon_max
    float lat_min
    float lat_max
    
cpdef int in_range_c(object point, object rect):
    cdef float lon, lat
    cdef Rect r1
    lon, lat = point
    r1.lon_min, r1.lon_max, r1.lat_min, r1.lat_max = rect
    if lon < r1.lon_min or lon > r1.lon_max:
        return False
    if lat < r1.lat_min or lat > r1.lat_max:
        return False
    return True

In [3]:
from glob import glob
flickr_datas = glob('*.txt')
poi_data = 'POI.csv'
# read poi info
pois = {}
with open(poi_data) as f:
    for line in f:
        l = line.strip().split(',')
        pois[l[-1].encode()] = [float(i) for i in l[:-1]]
for k, v in pois.items():
    pois[k] = sorted([v[1], v[3]]) + sorted([v[0], v[2]]) # [lon_min, lon_max, lat_min, lat_max]]

In [4]:
for file in flickr_datas:
    with open(file, 'rb') as f, open('total.csv', 'ab') as g:
        for line in f:
            coordinates = [float(i) for i in line.split(b'\t')[7:9]] # [lon, lat]
            for site, rect in pois.items():
                if in_range_c(coordinates, rect):
                    g.write(site + b'\t' + line)

In [5]:
file_dict = {}
with open('total.csv', encoding='utf-8') as f:
    for line in f:
        site = line.split('\t')[0]
        if site not in file_dict:
            file_dict[site] = open('{}.site'.format(site), 'w', encoding='utf-8')
        file_dict[site].write(line.split('\t', 1)[1])
for site, file in file_dict.items():
    file.close()